# Spectral Gap Analysis for Random 3-Regular Graphs

**Research Goal:** Analyze the minimum energy gap (Δ_min) of the Adiabatic Quantum Computing (AQC) Hamiltonian H(s) for Max-Cut on 3-regular graphs.

This notebook will generate data to study the connection between Δ_min and QAOA performance, where AQC runtime scales as **T ∝ 1/(Δ_min)²**.


In [ ]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh
import pandas as pd
import time
from typing import List, Tuple

print("=" * 70)
print("  AQC SPECTRAL GAP ANALYSIS FOR RANDOM 3-REGULAR GRAPHS")
print("=" * 70)


## 1. Configuration Parameters


In [ ]:
N_QUBITS = 10          # Maximum number of qubits (nodes in the graph)
NUM_GRAPHS = 200       # Number of random 3-regular graph instances
S_RESOLUTION = 200     # Number of points to sample along s ∈ [0, 1]

OUTPUT_FILENAME = f'Delta_min_3_regular_N{N_QUBITS}_{NUM_GRAPHS}graphs.csv'

if N_QUBITS % 2 != 0:
    print(f"⚠️  WARNING: N={N_QUBITS} is odd. 3-regular graphs require N to be even.")
    
print(f"✓ Configuration:")
print(f"  • N_QUBITS: {N_QUBITS}")
print(f"  • NUM_GRAPHS: {NUM_GRAPHS}")
print(f"  • S_RESOLUTION: {S_RESOLUTION}")
print(f"  • Hilbert space: 2^{N_QUBITS} = {2**N_QUBITS}")
print(f"  • Output: {OUTPUT_FILENAME}")


## 2. Pauli Matrices and Hamiltonian Construction


In [ ]:
# Define Pauli matrices
SIGMA_X = np.array([[0, 1], [1, 0]], dtype=complex)
SIGMA_Z = np.array([[1, 0], [0, -1]], dtype=complex)
IDENTITY = np.eye(2, dtype=complex)

def pauli_tensor_product(op_list: List[np.ndarray]) -> np.ndarray:
    """Computes tensor product of 2x2 matrices using np.kron."""
    result = op_list[0]
    for op in op_list[1:]:
        result = np.kron(result, op)
    return result

def get_pauli_term(N: int, pauli_type: str, index1: int, index2: int = -1) -> np.ndarray:
    """Generates N-qubit operator (X_i or Z_i⊗Z_j) as 2^N × 2^N matrix."""
    operators = [IDENTITY] * N
    
    if pauli_type == 'X':
        operators[index1] = SIGMA_X
    elif pauli_type == 'ZZ':
        operators[index1] = SIGMA_Z
        operators[index2] = SIGMA_Z
    
    return pauli_tensor_product(operators)

def build_H_initial(N: int) -> np.ndarray:
    """Builds Initial Hamiltonian: H_initial = -∑ᵢ X̂ᵢ (transverse field)."""
    H_B = np.zeros((2**N, 2**N), dtype=complex)
    for i in range(N):
        H_B += get_pauli_term(N, 'X', i)
    return -H_B

def build_H_problem(N: int, edges: List[Tuple[int, int]]) -> np.ndarray:
    """Builds Problem Hamiltonian: H_problem = ∑₍ᵢ,ⱼ₎∈E ẐᵢẐⱼ (Max-Cut)."""
    H_P = np.zeros((2**N, 2**N), dtype=complex)
    for u, v in edges:
        H_P += get_pauli_term(N, 'ZZ', u, v)
    return H_P

def get_aqc_hamiltonian(s: float, H_B: np.ndarray, H_P: np.ndarray) -> np.ndarray:
    """Returns H(s) = (1-s)·H_initial + s·H_problem."""
    return (1 - s) * H_B + s * H_P

print("✓ Hamiltonian construction functions defined")


## 3. Spectral Gap Calculation Function


In [ ]:
def calculate_min_gap(H_B: np.ndarray, H_P: np.ndarray, s_points: np.ndarray) -> Tuple[float, float]:
    """
    Calculates minimum spectral gap Δ_min = min_s[E₁(s) - E₀(s)].
    
    Uses scipy.linalg.eigh with subset_by_index=(0,1) for optimal performance.
    
    Returns:
        Tuple of (min_gap, s_at_min_gap)
    """
    min_gap = np.inf
    s_at_min = 0.0
    
    for s in s_points:
        H_s = get_aqc_hamiltonian(s, H_B, H_P)
        
        # Compute only the two lowest eigenvalues (E₀, E₁)
        # This is MUCH faster than computing all eigenvalues
        eigenvalues = eigh(H_s, eigvals_only=True, subset_by_index=(0, 1))
        
        gap = eigenvalues[1] - eigenvalues[0]
        
        if gap < min_gap:
            min_gap = gap
            s_at_min = s
    
    return float(min_gap), float(s_at_min)

print("✓ Spectral gap calculation function defined")


## 4. Main Analysis Loop


In [ ]:
# Initialize
s_points = np.linspace(0.0, 1.0, S_RESOLUTION)
data = []

# Pre-build H_initial (same for all graphs)
print(f"🔨 Building H_initial matrix ({2**N_QUBITS}×{2**N_QUBITS})...")
H_B = build_H_initial(N_QUBITS)
print("   ✓ Done\n")

# Start analysis
start_time = time.time()
print(f"🚀 Processing {NUM_GRAPHS} random 3-regular graphs...")
print("-" * 70)

for i in range(NUM_GRAPHS):
    # Generate random 3-regular graph
    try:
        G = nx.random_regular_graph(d=3, n=N_QUBITS)
    except nx.NetworkXError as e:
        print(f"❌ Error generating graph {i+1}: {e}")
        continue
    
    edges = list(G.edges())
    
    # Build problem Hamiltonian for this graph
    H_P = build_H_problem(N_QUBITS, edges)
    
    # Calculate minimum spectral gap
    delta_min, s_min = calculate_min_gap(H_B, H_P, s_points)
    
    # Store results
    data.append({
        'N': N_QUBITS,
        'Graph_ID': i + 1,
        'Delta_min': delta_min,
        's_at_min': s_min,
        'Edges': str(edges)
    })
    
    # Progress report every 10 graphs
    if (i + 1) % 10 == 0:
        elapsed = time.time() - start_time
        avg_time = elapsed / (i + 1)
        eta = avg_time * (NUM_GRAPHS - i - 1)
        print(f"  [{i+1:3d}/{NUM_GRAPHS}] Δ_min={delta_min:.6f} at s={s_min:.3f} | "
              f"Time: {elapsed:.1f}s | ETA: {eta:.1f}s")

total_time = time.time() - start_time
print("-" * 70)
print(f"\n✅ Analysis complete in {total_time:.2f}s ({total_time/60:.2f} min)")


## 5. Save Results and Display Statistics


In [ ]:
# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv(OUTPUT_FILENAME, index=False)

# Display statistics
print(f"💾 Results saved to: {OUTPUT_FILENAME}\n")
print("📈 STATISTICS:")
print(f"   • Graphs processed: {len(data)}")
print(f"   • Mean Δ_min: {df['Delta_min'].mean():.6f}")
print(f"   • Std Δ_min: {df['Delta_min'].std():.6f}")
print(f"   • Min Δ_min: {df['Delta_min'].min():.6f}")
print(f"   • Max Δ_min: {df['Delta_min'].max():.6f}")
print(f"   • Median Δ_min: {df['Delta_min'].median():.6f}")

# Display first few rows
print(f"\n📋 First 5 rows of data:")
df.head()


## 6. (Optional) Visualization

Visualize the distribution of minimum spectral gaps across the ensemble.


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of Δ_min
axes[0].hist(df['Delta_min'], bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Δ_min', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title(f'Distribution of Minimum Spectral Gaps (N={N_QUBITS})', fontsize=14)
axes[0].grid(alpha=0.3)

# Histogram of s_at_min
axes[1].hist(df['s_at_min'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('s at Δ_min', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Distribution of s where Δ_min occurs', fontsize=14)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📊 Mean s at minimum gap: {df['s_at_min'].mean():.3f}")
